<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [7]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\saliu\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<00:00, 8.20MB/s]


### Import the required libraries

In [1]:
import pandas as pd

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [7]:
import os

print(os.getcwd())

C:\Users\saliu


In [19]:
data = pd.read_csv('ElectricCarData.csv')

In [21]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [25]:
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ['Brand', 'Model', 'RapidCharge', 'PowerTrain', 'PlugType', 'BodyStyle', 'Segment']

label_encoders = {}
for col in columns_to_encode:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

print(data.sample(10))


    Brand  Model  AccelSec  TopSpeed_KmH  Range_Km  Efficiency_WhKm  \
84     18     22      10.0           140       330              273   
79     24     68       3.2           260       390              215   
40     30     49       3.8           250       515              184   
44     26     45      12.3           130       195              166   
78      8     58       6.0           180       340              206   
90      1     93       4.5           210       320              270   
98     20      6       7.5           160       330              191   
11     20     39       7.9           144       220              164   
69      8     56       6.0           180       430              209   
30      1     90       6.8           190       280              231   

   FastCharge_KmH  RapidCharge  PowerTrain  PlugType  BodyStyle  Segment  \
84            290            1           1         2          5        6   
79            810            1           0         2          7   

## Split the dataset for training and testing in ratio 80:20

In [31]:
from sklearn.model_selection import train_test_split

X = data.drop('PriceEuro', axis=1)
y = data['PriceEuro']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape[0])
print(X_test.shape[0])


82
21


## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [49]:
from sklearn.tree import DecisionTreeRegressor

data.replace('-', pd.NA, inplace=True)

data = data.apply(pd.to_numeric, errors='coerce')

data.dropna(inplace=True)

regressor = DecisionTreeRegressor(max_depth=5, min_samples_split=10, max_features='sqrt', random_state=42)
regressor.fit(X_train, y_train)

print("Model trained")


Model trained


## Predict the outcomes for X test

In [51]:
y_pred = regressor.predict(X_test)

print(y_pred[:10])

[ 54772.8         86345.          77933.33333333  33240.2
  33240.2         71614.33333333 151020.5         71614.33333333
  54772.8         86345.        ]


## Assess the model performance, by using sklearn metrics for regression

In [59]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)

r2 = r2_score(y_test, y_pred)

print(f'{mae:.2f}')
print(f'{mse:.2f}')
print(f'{rmse:.2f}')
print(f'{r2:.2f}')


11522.79
320270818.76
17896.11
0.54


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [82]:
from xgboost import XGBRegressor

xgb_regressor = XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    objective='reg:squarederror',
    random_state=42
)


xgb_regressor.fit(X_train, y_train)

print("XGBRegressor model trained")

XGBRegressor model trained


## Predict the outcomes for X test

In [80]:
y_pred = xgb_regressor.predict(X_test)

print(y_pred)

[68486.43  98919.805 93047.766 36278.867 37877.38  67516.09  95424.35
 74133.19  63336.523 68401.56  39122.97  59437.664 97432.086 35158.863
 24412.7   36144.52  34212.402 35592.176 39282.96  33915.99 ]


## Assess the model performance, by using sklearn metrics for regression

In [78]:
y_pred = xgb_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'{mae:.2f}')
print(f'{mse:.2f}')
print(f'{rmse:.2f}')
print(f'{r2:.2f}')

5614.06
52808744.49
7266.96
0.92


## Compare the performances of both model for at least three regression metircs

In [86]:
y_pred_dt = regressor.predict(X_test)

mae_dt = mean_absolute_error(y_test, y_pred_dt)
mse_dt = mean_squared_error(y_test, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)
r2_dt = r2_score(y_test, y_pred_dt)

y_pred_xgb = xgb_regressor.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("DecisionTreeRegressor Performance:")
print(f"MAE: {mae_dt}")
print(f"MSE: {mse_dt}")
print(f"RMS: {rmse_dt}")
print(f"R²: {r2_dt}")

print("\nXGBRegressor Performance:")
print(f"MAE: {mae_xgb}")
print(f"MSE: {mse_xgb}")
print(f"RMSE: {rmse_xgb}")
print(f"R²: {r2_xgb}")


DecisionTreeRegressor Performance:
MAE: 11522.793333333331
MSE: 320270818.7596667
RMS: 17896.111833570627
R²: 0.5413057824070179

XGBRegressor Performance:
MAE: 5614.0642578125
MSE: 52808744.48911057
RMSE: 7266.962535276383
R²: 0.9243669509887695
